In [11]:
import pandas as pd
import re
from csv import writer

In [12]:
colnames=['Name','Date','Time', 'Text','Cleaned_text'] 
exchanges = ['BINANCE', 'BYBIT', 'FTX', 'HUOBI', 'KUCOIN', 'POLONIEX','KRAKEN']
df = pd.read_csv('./1_Dash2Trade.csv',names = colnames,header=None)
print(df['Cleaned_text'][0])
print(df['Text'][0])
df.head()

['Long', 'KAVAUSDT', 'LEVERAGE', 'Isolated', '10XENTRY', '1.538', '-', '1.521TP1', '1.569TP2', '1.601TP3', '1.631TP4', '1.661STOP', '1.484Use', 'Trailing', 'SL', 'profit']
Long : KAVA/USDT LEVERAGE Isolated 10XENTRY 1.538 - 1.521TP1 1.569TP2 1.601TP3 1.631TP4 1.661STOP 1.484(Use Trailing SL when in profit)


,Name,Date,Time,Text,Cleaned_text
0,Dash2Trade,2022-09-15,05:37:14,Long : KAVA/USDT LEVERAGE Isolated 10XENTRY 1....,"['Long', 'KAVAUSDT', 'LEVERAGE', 'Isolated', '..."
1,Dash2Trade,2022-09-15,05:39:30,Long : KAVA/USDT LEVERAGE Isolated 10X ENTR...,"['Long', 'KAVAUSDT', 'LEVERAGE', 'Isolated', '..."
2,Dash2Trade,2022-09-15,06:12:05,Long : KAVA/USDT LEVERAGE Isolated 10X ENTR...,"['Long', 'KAVAUSDT', 'LEVERAGE', 'Isolated', '..."
3,Dash2Trade,2022-09-15,07:15:01,Long : KAVA/USDT LEVERAGE Isolated 10X ENTR...,"['Long', 'KAVAUSDT', 'LEVERAGE', 'Isolated', '..."
4,Dash2Trade,2022-09-15,15:27:57,"ByBit USDT, Binance Futures#COMP/USDT Entry ta...","['ByBit', 'USDT', 'Binance', 'FuturesCOMPUSDT'..."


In [23]:
exchanges = 'Binance|kucoin|bitbuy'
conversion = 'usdt|usd|btc|eth'
exchangeregex = '|'.join(exchanges)
target = ('|').join(['1','2','3'])
# coins=''
def extractor(i):
  text = re.sub('[\',\[\]]', '', i)

  if(re.search('signal|leverage',text,re.IGNORECASE) and re.search('target|targets',text,re.IGNORECASE)):    

    coin_name=re.findall('(\w+)(?:usdt|usd|btc|eth)',text,re.IGNORECASE)

    conversion_name = re.findall('\w+({})'.format(conversion),str(text),re.IGNORECASE)

    """exchange not used"""
    call_exchange = re.findall('\w+({})'.format(exchanges),text,re.IGNORECASE)

    """call_type not used"""
    call_type = re.findall('sell|buy',text,re.IGNORECASE)

    call_duration = re.findall('long|short',text,re.IGNORECASE)

    # targets = re.findall("(\d*[.]?\d*)[ ]?(-)?[ ]?(\d*[.]?\d*)[ ]?-[ ]?(\d*[.]?\d*)[ ]?-[ ]?(\d*[.]?\d*)[ ]?-[ ]?(\d*[.]?\d*)",text,re.IGNORECASE)
    """some targets are not extracted although format is same """
    T={}
    # targets = re.findall("Target[s]*[ ]*(.*?)[a-zA-Z]",text,re.IGNORECASE)
    targets=re.findall(r'targets\s*\d\s*(\d.\d*)\s*\d\s*(\d.\d*)\s*\d\s*(\d.\d*)\s*\d\s*(\d.\d*)',text,re.IGNORECASE)
        
    
    if targets:
      for x in range(len(targets[0])):
        T['T'+str(x+1)]=targets[0][x]

    leverage = re.findall('[ ]*[-]*[ ]*(\d+)x',text,re.IGNORECASE)

    """ stop loss not used that much"""
    stop_loss = re.findall('stop\s*-*\s*loss\s*-*\s*(\d.\d*)',text,re.IGNORECASE)

    with open('./2_Dash2Trade.csv', 'a',newline='') as f_object:
      writer_object = writer(f_object)
      ls = [coin_name,conversion_name,leverage,call_exchange,call_type,call_duration,T,stop_loss]
      writer_object.writerow(ls)
      f_object.close()

  return i
  
df['Cleaned_text'].apply(extractor)

UnicodeEncodeError: 'charmap' codec can't encode character '\U0001d7ed' in position 95: character maps to <undefined>